In [11]:
from geopy.distance import geodesic
import folium

In [12]:
# input data
pos_list = [
    [40.6643, -73.9385, "New York"], #departure
    [38.9041, -77.0171, "Washington"], 
    [42.332, -71.0202, "Boston"],
    [41.8379, -87.6828, "Chicago"],
    [45.4208, -75.6945, "Otawwa"],
    [43.7166, -79.3407, "Tront"], 
    [46.8127, -71.2199, "Quebec"]
]

In [13]:
# calculate distance from latitude and longitude
def calc_distance(coord1: list, coord2: list) -> float:
    return geodesic(coord1, coord2).kilometers

In [14]:
# create distance matrix
def create_distance_matrix(locations: list) -> list:
    create_distance_matrix = []
    for from_node in locations:
        row = []
        for to_node in locations:
            # add 0 if from_node is equal to to_node
            if from_node == to_node:
                row.append(0)
            else:
                coord1 = [from_node[0], from_node[1]]
                coord2 = [to_node[0], to_node[1]]
                row.append(calc_distance(coord1, coord2))
        create_distance_matrix.append(row)
        
    return create_distance_matrix

In [15]:
# show the route as a text
def print_solution(manager, routing, solution):
    total_distance = 0
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        route_output =  'Route for agent {}:\n'.format(vehicle_id + 1)
        route_distance = 0

        while not routing.IsEnd(index):
            route_output += ' {} ->'.format(pos_list[manager.IndexToNode(index)][2])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            
            # add distance to route_distance if index is not the end
            if not routing.IsEnd(index):
                route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        
        route_output += ' {}\n'.format(pos_list[manager.IndexToNode(index)][2])
        route_output += 'Distance of the route: {} km\n'.format(route_distance)
        print(route_output)
        total_distance += route_distance
        
    print('Total distance of all routes: {} km'.format(total_distance))

In [16]:
# show results on the map
def visualize(manager, routing, solution):
    # map size
    f = folium.Figure(width=600, height=600)

    # create the map with center latitude and longitude
    center_lat = 40.000
    center_long = -75.000
    map = folium.Map(location=[center_lat, center_long], zoom_start=5)

    # the color of the line
    color = ["#0000ff", "#ff0000", "#008000"]

    # put pins on the map
    for pos in pos_list:
        folium.Marker([pos[0], pos[1]], popup=pos[2]).add_to(map)

    # draw lines between the places
    for vehicle_id in range(manager.GetNumberOfVehicles()):
        index = routing.Start(vehicle_id)
        first_place = pos_list[manager.IndexToNode(routing.Start(vehicle_id))]
        previous_index = index

        while not routing.IsEnd(index):
            if index != routing.Start(vehicle_id):
                previous_place = pos_list[manager.IndexToNode(previous_index)]
                current_place = pos_list[manager.IndexToNode(index)]

                # draw a line between the previous place and the current place
                folium.PolyLine(locations=[previous_place[:2], current_place[:2]],
                    weight=3,color=color[vehicle_id]).add_to(map)
                previous_index = index
        
            index = solution.Value(routing.NextVar(index))

        # from the last place to the first place
        folium.PolyLine(locations=[current_place[:2], first_place[:2]],
        weight=3,color=color[vehicle_id]).add_to(map)
                    
        f.add_child(map)

    return f

In [17]:
# create data model
def create_data_mode():
    data = {}
    data['distance_matrix'] = create_distance_matrix(pos_list)
    data['num_veicles'] = 2
    data['depot'] = 0
    return data

### create model

In [18]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# create model
data = create_data_mode()

# create index manager and routing model
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_veicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

distance_matrix = create_distance_matrix(pos_list)

# distance callback function
def callback_distance(from_idex, to_index):
    from_node = manager.IndexToNode(from_idex)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]

# minimize the distance cost
transit_callback_index = routing.RegisterTransitCallback(callback_distance)

routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# additional constraint
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0, # no slack
    3000, # max distance of each vehicle
    True, # start cumul to zero
    dimension_name    
)

distance_dimention = routing.GetDimensionOrDie(dimension_name)
# each vehicle should visit at least one node
for vehicle_id in range(data['num_veicles']):
    routing.solver().Add(distance_dimention.CumulVar(routing.Start(vehicle_id)) <=
                         distance_dimention.CumulVar(routing.End(vehicle_id)) - 3)

In [19]:
# create object
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# seek solution
solution = routing.SolveWithParameters(search_parameters)

SystemError: <built-in function RoutingModel_SolveWithParameters> returned a result with an exception set

In [ ]:
# print the result
if solution:
    print_solution(manager, routing, solution)
else:
    print("No solution found.")

Route for agent 1:
 New York -> Tront -> Chicago -> Washington -> New York
Distance of the route: 2234 km

Route for agent 2:
 New York -> Boston -> Quebec -> Otawwa -> New York
Distance of the route: 1182 km

Total distance of all routes: 3416 km


In [ ]:
# visualize the result
visualize(manager, routing, solution)

0
1
